# Modeling and visualizations

## Plotting with Julia

- [Plots.jl](https://github.com/JuliaPlots/Plots.jl): powerful and convenient visualization. Multiple backends.
- [PyPlot.jl](https://github.com/JuliaPy/PyPlot.jl): `matplotlib` in Julia.
- [Makie.jl](https://github.com/JuliaPlots/Makie.jl): a data visualization ecosystem for the Julia programming language, with high performance and extensibility. 

**References**

- [Plots.jl docs](http://docs.juliaplots.org/latest/)
- [Makie tutorials](https://makie.juliaplots.org/stable/tutorials/)

In [ ]:
using Plots

# Setup backend and default options (optional)
Plots.gr(fmt=:png, lw=2)

In [ ]:
f(x) = sin(sin(x) + 1)

In [ ]:
xs = 0.0:0.1:4pi

In [ ]:
ys = f.(xs)

In [ ]:
# Line plots connect the data points
plot(xs, ys)

In [ ]:
# scatter plots show the data points only
scatter(xs, ys)

In [ ]:
# you can plot functions directly
plot(f, xs)

In [ ]:
# plot a function with a range
plot(f, 0.0, 4pi)

In [ ]:
# Customizations
plot(f, xs, 
     label="My line", legend=:bottom, 
     title="My Title",  line=(:red, 3),
     xlim = (0.0, 5.0), ylim = (-1.0, 1.5),
     xlabel="time", ylabel="My Mood", border=:box)

Multiple series: each row is one observation; each column is a variable.

In [ ]:
f2(x) = cos(cos(x) + 1)

In [ ]:
y2 = f2.(xs)

In [ ]:
plot(xs, [ys y2])

In [ ]:
# Customizations
plot(xs, [f, f2], label=["f1" "f2"], lc=[:black :green], title="Two time series")

In [ ]:
# Building the plot in multiple steps
# in the object-oriented way (recommended)

xMin = 0.0
xMax = 4.0pi
p1 = plot(f, xMin, xMax, label="f1", lc=:black)
plot!(p1 , f2, xMin, xMax, label="f2", lc=:lightsalmon)
plot!(p1, title = "My title", legend=:outertop)

In [ ]:
# Parametric function plot

xₜ(t) = sin(t)
yₜ(t) = sin(2t)

plot(xₜ, yₜ, 0, 2π, leg=false, fill=(0,:orange))

In [ ]:
# Subplots

p1 = plot(f, xs)
p2 = plot(f2, xs)
plot(p1, p2)

In [ ]:
plot(p1, p2, layout=(2, 1))

# Solving differential equations in Julia

## Define your model

For example, the [SIR model](https://www.maa.org/press/periodicals/loci/joma/the-sir-model-for-spread-of-disease-the-differential-equation-model) is a simple model of spreading of an contagious disease:

```math
\begin{align}
\frac{d}{dt}s(t) &= - \beta s(t)i(t)  \\
\frac{d}{dt}i(t) &= \beta s(t)i(t)  - \gamma i(t)  \\
\frac{d}{dt}r(t) &= \gamma i(t)
\end{align}
```

``s(t)`` is the proportion of susceptible people, ``i(t)`` is the proportion of infectious people, and ``r(t)`` is the proportion of recovered (or removed) people.

In [ ]:
using Plots, DifferentialEquations
Plots.gr(fmt=:png, lw=2)

In [ ]:
function sir(u, p ,t)
	s, i, r = u
	β, γ = p
	v1 = β * s * i
	v2 = γ * i
	return [-v1, v1-v2, v2]
end

## Parameters, initital conditions, and simulation time

In [ ]:
p = (β = 1.0, γ = 0.3)
u0 = [0.99, 0.01, 0.00]  # s, i, r
tspan = (0.0, 20.0)      # Or 20.0 alone

## In house algorithms

### The forward Euler method

In each step of the (forward) [Euler method](https://en.wikipedia.org/wiki/Euler_method), the next state variables ($\vec{u}_{n+1}$) are accumulated by the product of the time step (dt) and the derivatives at the current state ($\vec{u}_{n}$):

$$
\vec{u}_{n+1} = \vec{u}_{n} + dt \cdot f(\vec{u}_{n}, t_{n})
$$

In [ ]:
# ODE stepper
step_euler(f, u, p, t, dt) = u .+ dt .* f(u, p, t)

In [ ]:
# in house ODE solver
function mysolve(f, u0, tspan, p; dt=0.1, stepper=step_euler)
    ts = tspan[1]:dt:tspan[end]
    us = zeros(length(ts), length(u0))
    us[1, :] .= u0
    for i in 1:length(ts)-1
        us[i+1, :] .= stepper(f, us[i, :], p, ts[i], dt)
    end
    return (t = ts, u = us)
end

In [ ]:
solFE = mysolve(sir, u0, tspan, p, dt=0.5, stepper=step_euler)

In [ ]:
plot(solFE.t, solFE.u, label=["S" "I" "R"], legend=:right, )

### The fourth order Runge-Kutta (RK4) method

One of the most popular ODE-solving methods is the fourth order Runge-Kutta ([RK4](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods)) method.

In each step, the next state is calculated in 5 steps with 4 intermediate steps.

$$
\begin{align}
k_1 &= dt \cdot f(\vec{u}_{n}, t_n)  \\
k_2 &= dt \cdot f(\vec{u}_{n} + 0.5k_1, t_n + 0.5dt)  \\
k_3 &= dt \cdot f(\vec{u}_{n} + 0.5k_2, t_n + 0.5dt)  \\
k_4 &= dt \cdot f(\vec{u}_{n} + k_3, t_n + dt)  \\
u_{n+1} &= \vec{u}_{n} + \frac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4)
\end{align}
$$

In [ ]:
step_rk4(f, u, p, t, dt) = """TODO"""

## Use `DifferentialEquations.jl` package

In [ ]:
prob = ODEProblem(sir, u0, tspan, p)

In [ ]:
sol = solve(prob)

In [ ]:
plot(sol, label=["S" "I" "R"], legend=:right)

# Gillespie Algorithm

In [ ]:
using StatsBase # Weights() and sample()
using Random    # randexp()
using Plots

Plots.gr(fmt=:png, lw=2)

In [ ]:
#=
Stochastic chemical reaction: Gillespie Algorithm (direct method)
Adapted from: Chemical and Biomedical Enginnering Calculations Using Python Ch.4-3
=#
function ssa_direct(model, u0, tend, p, stoich; tstart=zero(tend))
    t = tstart   # Current time
    ts = [t]     # Time points
    u = copy(u0) # Current state
    us = copy(u) # Record of states
    while t < tend
        a = model(u, p, t)               # propensities
        dt = randexp() / sum(a)          # Time step
        du = sample(stoich, Weights(a))  # Choose the stoichiometry for the next reaction
        u .+= du  # Update state
        t += dt   # Update time
        
        us = [us u]  # Append state variable to record
        push!(ts, t) # Append time point to record
    end
    # Trasnpose to make column as variables, rows as observations
    us = collect(us')
    return (t = ts, u = us)
end

In [ ]:
#=
Stochastic chemical reaction: Gillespie Algorithm (first reaction method)
Adapted from: Chemical and Biomedical Enginnering Calculations Using Python Ch.4-3
=#
function ssa_first(model, u0, tend, p, stoich; tstart=zero(tend))
    t = tstart   # Current time
    ts = [t]     # Time points
    u = copy(u0) # Current state
    us = copy(u) # Record of states
    while t < tend
        a = model(u, p, t)  # propensities of reactions
        # dts from all reactions
        dts = randexp(length(a)) ./ a
        # Choose the reaction 
        i = argmin(dts)
        dt = dts[i]
        du = stoich[i]
        # Update state and time
        u .+= du
        t += dt
        us = [us u]  # Append state variable to record
        push!(ts, t) # Append time point to record
    end
    # Make column as variables, rows as observations
    us = collect(us')
    return (t = ts, u = us)
end

In [ ]:
#=
Reaction of A <-> B with rate constants k1 & k2
=#
"Propensity model for this reaction"
model(u, p, t) = [p.k1 * u[1],  p.k2 * u[2]]

In [ ]:
parameters = (k1=1.0, k2=0.5, stoich=[[-1, 1], [1, -1]])
u0 = [200, 0]
tend = 10.0

soldirect = ssa_direct(model, u0, tend, parameters, parameters.stoich)
solfirst = ssa_first(model, u0, tend, parameters, parameters.stoich)

In [ ]:
plot(soldirect.t, soldirect.u, xlabel="time", ylabel="# of molecules", title = "SSA (direct method)", label=["A" "B"])

In [ ]:
plot(solfirst.t, solfirst.u, xlabel="time", ylabel="# of molecules", title = "SSA (1st reaction method)", label=["A" "B"])

In [ ]:
# Running an ensemble of simulations
numRuns = 50
sol = ssa_direct(model, u0, tend, parameters, parameters.stoich)
p = plot(sol.t, sol.u, linecolor=[:blue :red], label=["A" "B"], linealpha=0.2,
    xlabel="time", ylabel="# of molecules", 
    title = "SSA (1st reaction method) ensemble")
for i in 1:numRuns-1
    sol = ssa_direct(model, u0, tend, parameters, parameters.stoich)
    plot!(p, sol.t, sol.u, linecolor=[:blue :red], linealpha=0.2, label=false)
end

p